In [6]:
import pandas as pd

data_root = './data/'
train_data=pd.read_csv(data_root+'train.tsv',sep='\t')
test_data=pd.read_csv(data_root+'test.tsv',sep='\t')
pd.set_option('display.width', 900)
print(train_data)

Phrases = train_data['Phrase']
MaxSeqlen = 0
for p in Phrases:
    words = p.split()
    if len(words) > MaxSeqlen:
        MaxSeqlen = len(words)
MaxSeqlen

PhraseId  SentenceId                                             Phrase  Sentiment
0              1           1  A series of escapades demonstrating the adage ...          1
1              2           1  A series of escapades demonstrating the adage ...          2
2              3           1                                           A series          2
3              4           1                                                  A          2
4              5           1                                             series          2
...          ...         ...                                                ...        ...
156055    156056        8544                                          Hearst 's          2
156056    156057        8544                          forced avuncular chortles          1
156057    156058        8544                                 avuncular chortles          3
156058    156059        8544                                          avuncular          2
156059 

52

In [27]:
import numpy as np

word2id = {}

BOW = np.zeros((len(Phrases),MaxSeqlen),dtype=np.float)
for i,p in enumerate(Phrases):
    words = p.split()
    words_id = []
    for word in words:
        if word not in word2id:
            word2id[word]=len(word2id)
        words_id.append(word2id[word])
    BOW[i][:len(words)] = words_id

print(BOW[:20])
labels = train_data['Sentiment']
label_list = list(set(labels))

[[ 0.  1.  2. ...  0.  0.  0.]
 [ 0.  1.  2. ...  0.  0.  0.]
 [ 0.  1.  0. ...  0.  0.  0.]
 ...
 [ 8.  9. 10. ...  0.  0.  0.]
 [ 8.  0.  0. ...  0.  0.  0.]
 [ 9. 10. 11. ...  0.  0.  0.]]
(156060, 5)


## Softmax regression

In [39]:
n_iter = 10
batch_size = 256
n_features = MaxSeqlen
n_class = len(label_list)
lr = 1e-3
print_every = 50

def make_network(n_hidden=128):
    model = dict(
        W1=np.random.randn(MaxSeqlen, n_hidden),
        W2=np.random.randn(n_hidden, n_class)
    )
    return model

def softmax(x):
    return np.exp(x)/(np.sum(np.exp(x),axis=1)[:,None])

def forward(x,w):
    h = x @ w['W1']
    x = h @ w['W2']
    output = softmax(x)
    return h, output

def ce_loss(probs, labels):
    log_probs = np.log(probs)
    return -np.sum(log_probs[:,labels])

def backward(loss, h, w, err):
    dW2 = h.T @ err

model = make_network()
step = 0
for i in range(n_iter):
    start_idx = 0
    while start_idx + batch_size <= BOW.shape[0]:
        train_data = BOW[start_idx:start_idx+batch_size]
        train_label = labels[start_idx:start_idx+batch_size]
        h, output = forward(train_data, model)
        loss = ce_loss(output, train_label)
        backward(loss, model)
        if step%print_every == 0:
            print("Step %d, loss: %.2f", loss)
        step += 1
        start_idx += batch_size

TypeError: invalid number of arguments